# การวิเคราะห์สัญญาณดิจิทัล

## วัตถุประสงค์
วิเคราะห์ความต้องการในการใช้แบนด์วิธด์ของสัญญาณแบบดิจิทัลแต่ละแบบ รวมถึงผลกระทบที่เกิดขึ้นเมื่อส่งสัญญาณผ่านตัวกลางที่มีแบนด์วิธด์จำกัด

## การเตรียมตัว
เรียกใช้ชุดคำสั่งด้านล่างเพื่อเตรียมสภาพแวดล้อมการโปรแกรมและโหลดไลบรารีที่จำเป็นดังนี้

In [1]:
from sigproc import Signal, start_notebook
bkp = start_notebook()

Loading BokehJS ...

Notebook session ready.


## สร้างสัญญาณดิจิทัล
ใช้ฟังก์ชันที่เตรียมไว้ให้ในโมดูล `digital.py` เพื่อสร้างสัญญาณจากบิตข้อมูลโดยใช้การเข้ารหัสสัญญาณดิจิทัลแบบต่าง ๆ

In [2]:
from digital import encode

data = '0100 1001 0011 1011'
# data = '1111111111111111111'
bitrate = 300   # บิตต่อวินาที

### วิเคราะห์สัญญาณที่ใช้การเข้ารหัสแบบ TTL

In [3]:
sig_ttl = encode('ttl',bitrate,data)
sig_ttl.plot_time()
sig_ttl.plot_freq()

### วิเคราะห์สัญญาณที่ใช้การเข้ารหัสแบบ NRZ (nonreturn to zero)

In [4]:
sig_nrz = encode("nrz",bitrate,data)
sig_nrz.plot_time()
sig_nrz.plot_freq()

### วิเคราะห์สัญญาณที่ใช้การเข้ารหัสแบบ RZ (return to zero)

In [5]:
sig_rz = encode("rz",bitrate,data)
sig_rz.plot_time()
sig_rz.plot_freq()

### วิเคราะห์สัญญาณที่ใช้การเข้ารหัสแบบ Manchester

In [6]:
sig_manchester = encode("manchester",bitrate,data)
sig_manchester.plot_time()
sig_manchester.plot_freq()

### วิเคราะห์สัญญาณที่ใช้การเข้ารหัสแบบ 2B1Q (2-Binary, 1-Quaternary)

In [7]:
sig_2b1q = encode("2b1q",bitrate,data)
sig_2b1q.plot_time()
sig_2b1q.plot_freq()

## แบนด์วิธด์ของสัญญาณและแบนด์วิธด์ของสื่อนำสัญญาณ

จะเห็นได้จากหัวข้อที่แล้วว่าสัญญาณที่เข้ารหัสแบบดิจิทัลจะมีองค์ประกอบของสัญญาณซายน์ที่มีความถี่ตั้งแต่ 0 Hz ไปจนถึงความถี่อนันต์ อย่างไรก็ตามไม่มีสื่อนำสัญญาณใดที่สามารถส่งผ่านความถี่แบบไม่จำกัดเช่นนี้ได้ หัวข้อนี้จะให้ศึกษาพฤติกรรมของสัญญาณเมื่อถูกส่งผ่านสื่อนำสัญญาณที่มีแบนด์วิธด์จำกัด ซึ่งหมายถึงสื่อที่ยอมให้คลื่นซายน์ในย่านความถี่ย่านหนึ่งผ่านได้ ส่วนความถี่อื่น ๆ จะผ่านไปไม่ได้หรือเล็ดลอดไปได้ด้วยการลดทอนที่สูง

ทดลองคัดลอกสัญญาณที่สร้างขึ้นด้วยการเข้ารหัสข้างต้นแล้วใช้เมท็อด `filter()` เพื่อเลือกเฉพาะความถี่ที่ต่ำกว่าค่าที่กำหนดลงมา

In [8]:
# CUTOFF_FREQ = 500
CUTOFF_FREQ = bitrate/2

signal = sig_nrz.copy()
signal_filtered = signal.copy()
signal_filtered.filter(CUTOFF_FREQ)

ตรวจสอบผลลัพธ์ที่เกิดขึ้นกับสัญญาณโดยเปรียบเทียบกับสัญญาณดั้งเดิม รวมถึงพล็อตจุด ณ ตำแหน่งกึ่งกลางของช่วงบิตเพื่อตรวจสอบค่า

In [9]:
import numpy as np

# fig_f=dict(height=200,x_range=(0,2000),y_range=(0,0.5))
# signal.plot_freq(fig_options=fig_f)
# signal_filtered.plot_freq(fig_options=fig_f)

fig=bkp.figure(height=200)

signal.plot_time(fig=fig,line_options=dict(color="blue"))
signal_filtered.plot_time(fig=fig,line_options=dict(color="red",line_width=1))

bit_interval = 1/bitrate
points = np.arange(0,signal.duration,bit_interval)+bit_interval/2
ft = signal_filtered.time_function
fig.scatter(x=points*1000,y=ft(points),size=5,color="green") 
bkp.show(fig)
print(data)

0100 1001 0011 1011


## ทดลองด้วยตนเอง
* เปลี่ยน bitrate ของสัญญาณให้สูงขึ้น เช่นเปลี่ยนเป็น 500 บิตต่อวินาที
* จำกัดแบนด์วิดธ์กับการเข้ารหัสสัญญาณรูปแบบอื่น ๆ นอกเหนือจาก NRZ
* จำกัดแบนด์วิดธ์สำหรับความถี่ต่ำ เพื่อจำลองการส่งสัญญาณผ่านช่องสัญญาณที่มีสมบัติเป็น high-pass filter
* ปรับเปลี่ยนบิตข้อมูลเพื่อให้สัญญาณมีคุณสมบัติดังนี้
  * เกิด DC component ขึ้นเป็นปริมาณมาก
  * เกิดสภาวะที่ระดับสัญญาณไม่เปลี่ยนแปลง ทำให้เสี่ยงต่อการเกิด synchronization loss
  * มีแบนด์วิธด์ของสัญญาณ*สูงที่สุด*เท่าที่จะเป็นไปได้
  * มีแบนด์วิธด์ของสัญญาณ*ต่ำที่สุด*เท่าที่จะเป็นไปได้